In [20]:
# Packages
import pandas as pd
import numpy as np
from sklearn.pipeline        import Pipeline
from sklearn.preprocessing   import FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics         import make_scorer
from sklearn.base            import clone
import joblib, mlflow
import mlflow.sklearn
import optuna
from optuna.integration import MLflowCallback
from catboost import CatBoostRegressor
from pathlib import Path
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
from scipy.stats import ttest_rel
from scipy.optimize import minimize_scalar, minimize

# Custom Metric for Training Feedback
def rmsle_xgb(preds, dtrain):
    y_true = dtrain.get_label()
    preds = np.maximum(preds, 0)
    rmsle = np.sqrt(np.mean((np.log1p(preds) - np.log1p(y_true)) ** 2))
    return 'rmsle', rmsle

# Custom Metric for GridSearch (wrapped in make_scorer)
def rmsle_sklearn(y_true, y_pred):
    y_pred = np.maximum(y_pred, 0)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))

rmsle_scorer = make_scorer(rmsle_sklearn, greater_is_better=False)

# Data
df = pd.read_csv('playground-series-s5e5/train.csv')
y = df['Calories']
X = df.drop(columns=(['Calories', 'id']))

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X['Sex']
)

# Custom Feature Engineering
def add_bmi_intensity(X_df: pd.DataFrame) -> pd.DataFrame:
    """Adds BMI and HeartRatexDuration features"""
    X = X_df.copy()
    X['BMI'] = (X['Weight'] / (X['Height'] / 100) ** 2).round(2)
    X['Timed_Intensity'] = X['Duration'] * X['Heart_Rate']
    X['Heart_Rate_Zone'] = (
        X['Heart_Rate'] / (220 - X['Age'])
    ) * 100
    X['Mifflin_Jeor_BMR'] = np.where(
        X['Sex'] == 'male',
        (10 * X['Weight']) + (6.25 * X['Height']) - (5 * X['Age']) + 5,
        (10 * X['Weight']) + (6.25 * X['Height']) - (5 * X['Age']) - 161,
    )
    return X

feat_eng = FunctionTransformer(add_bmi_intensity, validate=False)

# ──────────────────────────────────────────────────────────────────────
# MODEL & PIPELINE
# ──────────────────────────────────────────────────────────────────────
cat_model = CatBoostRegressor(
        loss_function="RMSE",
        random_seed=42,
        verbose=False,
        cat_features=["Sex"]
)

pipe = Pipeline([
    ("feat_eng", feat_eng),
    ("model", cat_model)
])

# ──────────────────────────────────────────────────────────────────────
# GRID  (prefix params with model__)
# ──────────────────────────────────────────────────────────────────────
def cat_objective(trial):
    params = {
        "model__depth":               trial.suggest_int("model__depth", 5, 10),
        "model__iterations":          trial.suggest_int("model__iterations", 600, 1600),
        "model__learning_rate":       trial.suggest_float("model__learning_rate", 0.01, 0.2, log=True),
        "model__l2_leaf_reg":         trial.suggest_float("model__l2_leaf_reg", 1, 10, log=True),
        "model__bagging_temperature": trial.suggest_float("model__bagging_temperature", 0, 1),
    }

    score = cross_val_score(
                clone(pipe).set_params(**params),
                X_train, y_train,
                scoring=rmsle_scorer,
                cv=5,
                n_jobs=-1
            ).mean()
    return score

In [ ]:

# ------------------------------------------------------------------
# MLFLOW + OPTUNA LOOP  (same structure)
# ------------------------------------------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Calories-CB-Optuna-V3")
mlflow.sklearn.autolog(log_models=False)

study_cb = optuna.create_study(direction="maximize", study_name="cat_rmsle_1")

with mlflow.start_run(run_name="optuna_parent_cb"):
    mlflow_cb = MLflowCallback(
        tracking_uri=mlflow.get_tracking_uri(),
        metric_name="neg_rmsle_cv",
        mlflow_kwargs={"nested": True}
    )

    study_cb.optimize(cat_objective, n_trials=50, callbacks=[mlflow_cb], show_progress_bar=True)

    mlflow.log_params(study_cb.best_trial.params)
    mlflow.log_metric("best_neg_rmsle_cv", study_cb.best_value)

In [ ]:
# ──────────────────────────────────────────────────────────────────────
# GENERATING OOF PREDICTIONS AND A VAL SCORE
# ──────────────────────────────────────────────────────────────────────
FE_VERSION = 'v4_hrzone_bmr_cb'

best_params = {
    "model__depth":               10,
    "model__iterations":          1594,
    "model__learning_rate":       0.08318728723922084,
    "model__l2_leaf_reg":         3.7912206788501206,
    "model__bagging_temperature": 0.3000645110427449,
}

cat_pipe = clone(pipe).set_params(**best_params)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_table = []
oof_cb = np.empty(len(X))

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Calories-CB-OOF-FINAL")

with mlflow.start_run(run_name="cb_oof_fit",
                      tags={"fe_version": FE_VERSION}):
    
    for fold, (tr_idx, val_idx) in enumerate(tqdm(kf.split(X, y), total=kf.get_n_splits()), 1):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        fold_model = clone(cat_pipe).fit(X_tr, y_tr)
        preds = fold_model.predict(X_val)

        oof_cb[val_idx] = preds

        fold_rmsle = rmsle_sklearn(y_val, preds)
        mlflow.log_metric(f"fold{fold}_rmsle", fold_rmsle)

        fold_table.append({"fold": fold, "rmsle": fold_rmsle})
        clear_output(wait=True)
        display(pd.DataFrame(fold_table))

    cv_rmsle = np.mean([row["rmsle"] for row in fold_table])
    mlflow.log_metric("cv_rmsle", cv_rmsle)
    print(f"5-fold CV RMSLE: {cv_rmsle:.5f}")

    Path("oof").mkdir(exist_ok=True)
    np.save("oof/oof_cb.npy", oof_cb)
    mlflow.log_artifact("oof/oof_cb.npy", artifact_path="oof")

,fold,rmsle
0,1,0.060914
1,2,0.060836
2,3,0.060249
3,4,0.061157
4,5,0.060671


2025/05/31 12:52:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run cb_oof_fit at: http://127.0.0.1:5000/#/experiments/28/runs/6f5d9125b174428c8c819ec3f43388b4.
2025/05/31 12:52:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/28.


5-fold CV RMSLE: 0.06077


In [28]:
# ------------------------------------------------------------------
# helper: download a single artefact from MLflow into a temp folder
# ------------------------------------------------------------------
def load_np(run_id: str, artifact_path: str) -> np.ndarray:
    local_path = mlflow.artifacts.download_artifacts(
        run_id=run_id,
        artifact_path=artifact_path
    )
    return np.load(local_path)

def get_experiment_or_die(name: str):
    exp = mlflow.get_experiment_by_name(name)
    if exp is None:
        raise ValueError(f"No experiment called '{name}' in current tracking store.")
    return exp

# ------------------------------------------------------------------
# locate the runs that hold the OOF .npy files
# ------------------------------------------------------------------
client = mlflow.tracking.MlflowClient()

exp_xgb = get_experiment_or_die("Calories-XGB-OOF-FINAL-GOLDEN")
exp_cb  = get_experiment_or_die("Calories-CB-OOF-FINAL")
exp_nn  = get_experiment_or_die("Calories-NN-OOF-VAL-V2")      # NEW

run_xgb = client.search_runs(exp_xgb.experiment_id,
                             "tags.mlflow.runName = 'xgb_oof_fit'")[0]
run_cb  = client.search_runs(exp_cb.experiment_id,
                             "tags.mlflow.runName = 'cb_oof_fit'")[0]
run_nn  = client.search_runs(exp_nn.experiment_id,             # NEW
                             "tags.mlflow.runName = 'nn_oof_fit'")[0]

# ------------------------------------------------------------------
# download & load
# ------------------------------------------------------------------
oof_xgb = load_np(run_xgb.info.run_id, "oof/oof_xgb.npy")
oof_cb  = load_np(run_cb.info.run_id,  "oof/oof_cb.npy")
oof_nn  = load_np(run_nn.info.run_id,  "oof/oof_nn.npy")       # NEW

print("Loaded shapes  →",
      "XGB:", oof_xgb.shape,
      "CB :", oof_cb.shape,
      "NN :", oof_nn.shape)

Loaded shapes  → XGB: (750000,) CB : (750000,) NN : (750000,)


In [25]:
# ────────────────────────────────────────────────────────────────────
#  STACKING  ➜  meta-learner on top of the three base OOF predictions
# ────────────────────────────────────────────────────────────────────
import numpy as np, pandas as pd, joblib, mlflow
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from pathlib import Path

# -------------------------------------------------------------------
# 0.  Prepare meta-features  (make sure oof_xgb / oof_cb / oof_nn exist)
# -------------------------------------------------------------------
X_meta = np.vstack([oof_xgb, oof_cb, oof_nn]).T         # shape (n_samples, 3)
y_meta = y_true                                         # ground-truth targets

# 1-line RMSLE scorer (same as before)
def rmsle_np(y, y_hat):
    y_hat = np.maximum(y_hat, 0)
    return np.sqrt(np.mean((np.log1p(y_hat) - np.log1p(y)) ** 2))

rmsle_scorer = make_scorer(rmsle_np, greater_is_better=False)

# -------------------------------------------------------------------
# 1.  CV splitter for the meta-learner
#     (pass the *object* — not the generator — so the model is picklable)
# -------------------------------------------------------------------
bins = pd.qcut(y_meta, 4, labels=False)
skf  = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# -------------------------------------------------------------------
# 2.  RidgeCV meta-model
# -------------------------------------------------------------------
alphas = np.logspace(-4, 3, 100)          # wide search range
meta_model = RidgeCV(
    alphas      = alphas,
    scoring     = rmsle_scorer,
    cv          = skf,                    # ← the fix: pass the splitter, not .split(...)
    store_cv_values = False               # nothing extra to pickle
)

# -------------------------------------------------------------------
# 3.  Train, evaluate, log to MLflow
# -------------------------------------------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Calories-Stacked-Ridge")

with mlflow.start_run(run_name="ridge_meta"):
    meta_model.fit(X_meta, y_meta)

    # OOF evaluation (should be identical to training data here)
    oof_pred   = meta_model.predict(X_meta)
    oof_rmsle  = rmsle_np(y_meta, oof_pred)

    mlflow.log_metric("oof_rmsle", oof_rmsle)
    mlflow.log_param("best_alpha", float(meta_model.alpha_))

    # persist the sklearn model (pure pickle, no TF dependency)
    model_path = Path("meta_ridge.pkl")
    joblib.dump(meta_model, model_path)
    mlflow.log_artifact(model_path, artifact_path="model")

print(f"Saved stacked ridge ✅  –  OOF RMSLE: {oof_rmsle:.6f}")

C:\Users\afise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:2341: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(
2025/05/31 17:28:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run ridge_meta at: http://127.0.0.1:5000/#/experiments/31/runs/b24e8402eff04dcea20399e493aae388.
2025/05/31 17:28:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/31.


Saved stacked ridge ✅  –  OOF RMSLE: 0.059810


In [26]:
# ────────────────────────────────────────────────────────────────────
#  STACKING  ➜  meta-learner on top of XGB + CB   (no NN this time)
# ────────────────────────────────────────────────────────────────────
import numpy as np, pandas as pd, joblib, mlflow
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from pathlib import Path

# -------------------------------------------------------------------
# 0.  Prepare meta-features
# -------------------------------------------------------------------
# make sure oof_xgb and oof_cb are in memory
assert oof_xgb.shape == oof_cb.shape, "OOF vectors have different lengths"

X_meta = np.vstack([oof_xgb, oof_cb]).T          # (n_samples, 2)
y_meta = y_true                                  # ground-truth targets

def rmsle_np(y, y_hat):
    y_hat = np.maximum(y_hat, 0)
    return np.sqrt(np.mean((np.log1p(y_hat) - np.log1p(y)) ** 2))

rmsle_scorer = make_scorer(rmsle_np, greater_is_better=False)

# -------------------------------------------------------------------
# 1.  Stratified K-fold for RidgeCV
# -------------------------------------------------------------------
bins = pd.qcut(y_meta, 4, labels=False)
skf  = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# -------------------------------------------------------------------
# 2.  RidgeCV meta-model
# -------------------------------------------------------------------
alphas     = np.logspace(-4, 3, 100)      # wide α grid
meta_model = RidgeCV(
    alphas      = alphas,
    scoring     = rmsle_scorer,
    cv          = skf,                    # pass the splitter *object*
    store_cv_values=False
)

# -------------------------------------------------------------------
# 3.  Train, evaluate, log to MLflow
# -------------------------------------------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Calories-Stacked-XGB+CB")

with mlflow.start_run(run_name="ridge_meta_xgb_cb"):
    meta_model.fit(X_meta, y_meta)

    oof_pred  = meta_model.predict(X_meta)
    oof_rmsle = rmsle_np(y_meta, oof_pred)

    mlflow.log_metric("oof_rmsle", oof_rmsle)
    mlflow.log_param("best_alpha", float(meta_model.alpha_))
    mlflow.log_param("features",   "XGB,CB")

    # save the sklearn ridge (pure pickle, no TF deps)
    model_path = Path("meta_ridge_xgb_cb.pkl")
    joblib.dump(meta_model, model_path)
    mlflow.log_artifact(model_path, artifact_path="model")

print(f"Stacked ridge (XGB+CB) logged ✅   OOF RMSLE: {oof_rmsle:.6f}")

2025/05/31 17:38:10 INFO mlflow.tracking.fluent: Experiment with name 'Calories-Stacked-XGB+CB' does not exist. Creating a new experiment.
C:\Users\afise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:2341: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(
2025/05/31 17:38:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run ridge_meta_xgb_cb at: http://127.0.0.1:5000/#/experiments/32/runs/5a417207e81a4568b42161dbaee70ea1.
2025/05/31 17:38:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/32.


Stacked ridge (XGB+CB) logged ✅   OOF RMSLE: 0.060115


In [31]:
# ────────────────────────────────────────────────────────────────────
#  STACKING  ➜  meta-learner on top of XGB + NN
# ────────────────────────────────────────────────────────────────────
import numpy as np, pandas as pd, joblib, mlflow
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from pathlib import Path

# -------------------------------------------------------------------
# 0.  Prepare meta-features
# -------------------------------------------------------------------
# make sure oof_xgb and oof_nn are already in memory, and y_true is the target
assert oof_xgb.shape == oof_nn.shape, "OOF vectors have different lengths"

X_meta = np.vstack([oof_xgb, oof_nn]).T        # (n_samples, 2)
y_meta = y_true                                # ground-truth targets

def rmsle_np(y, y_hat):
    y_hat = np.maximum(y_hat, 0)
    return np.sqrt(np.mean((np.log1p(y_hat) - np.log1p(y)) ** 2))

rmsle_scorer = make_scorer(rmsle_np, greater_is_better=False)

# -------------------------------------------------------------------
# 1.  Stratified K-fold for RidgeCV
# -------------------------------------------------------------------
bins = pd.qcut(y_meta, 4, labels=False)
skf  = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# -------------------------------------------------------------------
# 2.  RidgeCV meta-model
# -------------------------------------------------------------------
alphas     = np.logspace(-4, 3, 100)           # wide α grid
meta_model = RidgeCV(
    alphas      = alphas,
    scoring     = rmsle_scorer,
    cv          = skf,
    store_cv_values=False
)

# -------------------------------------------------------------------
# 3.  Train, evaluate, log to MLflow
# -------------------------------------------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Calories-Stacked-XGB+NN")

with mlflow.start_run(run_name="ridge_meta_xgb_nn") as run:
    meta_model.fit(X_meta, y_meta)

    oof_pred  = meta_model.predict(X_meta)
    oof_rmsle = rmsle_np(y_meta, oof_pred)

    mlflow.log_metric("oof_rmsle", oof_rmsle)
    mlflow.log_param("best_alpha", float(meta_model.alpha_))
    mlflow.log_param("features",   "XGB,NN")

    # save the sklearn ridge model
    model_path = Path("meta_ridge_xgb_nn.pkl")
    joblib.dump(meta_model, model_path)
    mlflow.log_artifact(model_path, artifact_path="model")

print(f"Stacked ridge (XGB+NN) logged ✅   OOF RMSLE: {oof_rmsle:.6f}")
print("MLflow run_id:", run.info.run_id)

2025/05/31 18:32:03 INFO mlflow.tracking.fluent: Experiment with name 'Calories-Stacked-XGB+NN' does not exist. Creating a new experiment.
C:\Users\afise\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:2341: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(
2025/05/31 18:32:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run ridge_meta_xgb_nn at: http://127.0.0.1:5000/#/experiments/34/runs/f5b1c411b36148179f71aaa16677dc0f.
2025/05/31 18:32:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/34.


Stacked ridge (XGB+NN) logged ✅   OOF RMSLE: 0.059838
MLflow run_id: f5b1c411b36148179f71aaa16677dc0f


In [18]:
# ------------------------------------------------------------
# metrics & paired t-tests
# ------------------------------------------------------------
y_true = y.values          # ground truth used in OOF generation

def rmsle_vec(y, y_hat):
    return np.sqrt(np.mean((np.log1p(np.clip(y_hat, 0, None)) -
                            np.log1p(y))**2))

rmsle_xgb = rmsle_vec(y_true, oof_xgb)
rmsle_cb  = rmsle_vec(y_true, oof_cb)
rmsle_nn  = rmsle_vec(y_true, oof_nn)

print(f"OOF RMSLE  XGB : {rmsle_xgb:.6f}")
print(f"OOF RMSLE  CB  : {rmsle_cb :.6f}")
print(f"OOF RMSLE  NN  : {rmsle_nn :.6f}")

# per-row squared-log errors
sle_xgb = (np.log1p(np.clip(oof_xgb, 0, None)) - np.log1p(y_true))**2
sle_cb  = (np.log1p(np.clip(oof_cb , 0, None)) - np.log1p(y_true))**2
sle_nn  = (np.log1p(np.clip(oof_nn , 0, None)) - np.log1p(y_true))**2

def report_t(a, b, name_a, name_b):
    t_stat, p_val = ttest_rel(a, b)
    print(f"{name_a} vs {name_b} : ΔSLE t={t_stat:.2f}, p={p_val:.3g}")

report_t(sle_xgb, sle_cb, "XGB", "CB")
report_t(sle_xgb, sle_nn, "XGB", "NN")
report_t(sle_cb , sle_nn, "CB" , "NN")

OOF RMSLE  XGB : 0.060336
OOF RMSLE  CB  : 0.060766
OOF RMSLE  NN  : 0.060649
XGB vs CB : ΔSLE t=-4.51, p=6.43e-06
XGB vs NN : ΔSLE t=-1.53, p=0.126
CB vs NN : ΔSLE t=0.63, p=0.526


In [21]:
def rmsle_blend(w):
    blend = w * oof_xgb + (1 - w) * oof_cb
    return rmsle_vec(y_true, blend)

opt = minimize_scalar(rmsle_blend, bounds=(0, 1), method="bounded")
w_opt = opt.x
print(f"Optimal weight (XGB share) = {w_opt:.3f}")
print(f"Blended OOF RMSLE = {opt.fun:.6f}")

Optimal weight (XGB share) = 0.629
Blended OOF RMSLE = 0.060107


In [22]:
# three OOF vectors already in memory
y_true  = y.values               # ground-truth targets
oofs    = np.column_stack([oof_xgb, oof_cb, oof_nn])   # shape (n_rows, 3)

def rmsle_from_weights(w12):                     # w12 = [w_xgb, w_cb]
    w1, w2 = w12
    if w1 < 0 or w2 < 0 or w1 + w2 > 1:          # keep inside simplex
        return np.inf
    w3      = 1.0 - w1 - w2                      # weight for NN
    blend   = (oofs * [w1, w2, w3]).sum(axis=1)
    return rmsle_vec(y_true, blend)

# start at equal weights, constrain to [0,1] box-square
result = minimize(
    rmsle_from_weights,
    x0=[1/3, 1/3],
    bounds=[(0, 1), (0, 1)],
    method="L-BFGS-B"
)

w_xgb, w_cb = result.x
w_nn        = 1.0 - w_xgb - w_cb

print(f"Optimal weights  →  XGB {w_xgb:.3f},  CB {w_cb:.3f},  NN {w_nn:.3f}")
print(f"Blended OOF RMSLE = {result.fun:.6f}")

Optimal weights  →  XGB 0.399,  CB 0.174,  NN 0.426
Blended OOF RMSLE = 0.059716


In [23]:
print(np.corrcoef(oof_xgb, oof_cb)[0,1])   # ~0.93  (very similar)
print(np.corrcoef(oof_xgb, oof_nn)[0,1])   # ~0.88
print(np.corrcoef(oof_cb , oof_nn)[0,1])   # ~0.87

0.9999213467873572
0.99977438333897
0.9998097861747608


In [13]:
# FINAL-MODEL RUN  ──────────────────────────────────────────────
best_params = {
    "model__depth":               10,
    "model__iterations":          1594,
    "model__learning_rate":       0.08318728723922084,
    "model__l2_leaf_reg":         3.7912206788501206,
    "model__bagging_temperature": 0.3000645110427449,
}

cat_final = CatBoostRegressor(
        loss_function="RMSE",
        random_seed=42,
        verbose=False,
        cat_features=["Sex"]
)

final_pipe = Pipeline([
        ("feat_eng", feat_eng),
        ("model",    cat_final)
]).set_params(**best_params)

# ------------------------------------------------------------------
# Fit on the full dataset (train + val)
# ------------------------------------------------------------------
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

final_pipe.fit(X_full, y_full)

# ------------------------------------------------------------------
# Save / log the artefact
# ------------------------------------------------------------------
joblib.dump(final_pipe, "calories_catboost_optuna.joblib")

# Log to MLflow
with mlflow.start_run(run_name="catboost_final_fit_v2"):
    mlflow.log_params(best_params)
    mlflow.sklearn.log_model(final_pipe, artifact_path="model")

2025/05/31 13:05:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/31 13:05:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run catboost_final_fit_v2 at: http://127.0.0.1:5000/#/experiments/28/runs/4a9b55aebbce4b4f9d87ecd98a596d45.
2025/05/31 13:05:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/28.
